In [1]:
f = open('day20.txt')
input_puzzle = f.read()

f = open('day20_seamonster.txt')
sea_monster = [list(x) for x in f.read().split('\n')]

def codify_monster(sea_monster):
    idx = []
    for i in range(len(sea_monster)):
        for j in range(len(sea_monster[0])):
            if sea_monster[i][j] == '#':
                idx.append([i,j])
    return idx

idx_monster = codify_monster(sea_monster)

In [2]:
import numpy as np

def extract_tiles(string):
    dict_tiles = {}
    tiles = string.split('\n\n')
    for tile in tiles:
        dict_tiles[tile[5:9]] = np.array([list(x) for x in tile[11:].split('\n')])
        
    return dict_tiles
    
def count_sides(string, tile):
    count = 0
    
    for i in range(4):
        if string == ''.join(np.rot90(tile, k=i)[0]):
            count += 1
            
    for i in range(4):
        if string == ''.join(np.rot90(np.transpose(tile), k=i)[0]):
            count += 1
            
    return count

def get_borders(tile):
    borders = []
    
    for i in range(4):
        borders.append(''.join(np.rot90(tile, k=i)[0]))
            
    return borders

def print_tile(tile):
    for line in tile:
        print(''.join(line))
    print()

def count_border_fits(input_key, dict_tiles):
    count = 0
    tile = dict_tiles[input_key]
    
    for border in get_borders(tile):
        for key in dict_tiles.keys():
            if key != input_key:
                other_tile = dict_tiles[key]

                if count_sides(border, other_tile) > 0:
                    count += 1
                    break

    return count
        
dict_tiles = extract_tiles(input_puzzle)
# dict_frame = get_frame(dict_tiles)

# prod = 1
# for key in dict_tiles.keys():
#     if count_border_fits(key, dict_tiles) == 2:
#         print(key)
#         prod *= int(key)
        
# prod

In [13]:
from termcolor import colored

def get_frame(dict_tiles):
    dict_frame = {}
    for key in dict_tiles.keys():
        count = count_border_fits(key, dict_tiles)
#         if count < 4:
        dict_frame[key] = count    
        
    return dict_frame

def print_picture(picture):
    for picture_row in range(len(picture)):
        for picture_column in range(len(picture)):
            if picture[picture_row][picture_column] == 'O':
                print(colored('O', 'red'), end ='')
            else:
                print(picture[picture_row][picture_column], end ='')
        print('\n', end='')
            
def build_picture(picture_tiles):
    picture = []
    for picture_row in range(12):
        for tile_row in range(1,9):
            picture.append(''.join([''.join(picture_tiles[picture_row][picture_column][tile_row][1:-1]) for picture_column in range(12)]))
                
    return [list(x) for x in picture]


def match_tile_down(tile, keys):
    for key in keys:
        
        for r in range(4):
            for f in [False, True]:
                other_tile = orient_tile(dict_tiles[key], r, f)
                if ''.join(tile[-1]) == ''.join(other_tile[0]):
                    return key, r, f
       
def match_tile_right(tile, keys):
    for key in keys:
        
        for r in range(4):
            for f in [False, True]:
                other_tile = orient_tile(dict_tiles[key], r, f)
                if ''.join(np.rot90(tile, k = -1)[-1]) == ''.join(np.rot90(other_tile, k = -1)[0]):
                    return key, r, f

def orient_tile(tile, r, f):
    tile = np.rot90(tile, k=r)
    
    if f:
        tile = np.transpose(tile)
            
    return tile

def place_tiles(dict_tiles):
    picture_tiles = [[0]*12 for x in range(12)]
    picture_tiles[0][0] = np.rot90(dict_tiles['1091'], k = 2)
    keys = list(dict_tiles.keys())
    keys.remove('1091')

    for i in range(12):
        for j in range(12):
            if j < 11:
                key, r, f = match_tile_down(picture_tiles[j][i], keys)
                picture_tiles[j+1][i] = orient_tile(dict_tiles[key], r, f)
                keys.remove(key)

            if j == 11 and i != 11:
                key, r, f = match_tile_right(picture_tiles[0][i], keys)
                picture_tiles[0][i+1] = orient_tile(dict_tiles[key], r, f)
                keys.remove(key)
    
    return picture_tiles

def is_monster(picture, i, j):
    for idx in idx_monster:
        if picture[i+idx[0]][j+idx[1]] != "#":
            return False
    
    return True

def find_orientation(picture):
    check = False
    
    for r in range(4):
        for f in [False, True]:
            picture_test = orient_tile(picture, r, f)
            
            for i in range(len(picture_test)-len(sea_monster)):
                for j in range(len(picture_test)-len(sea_monster[0])):

                    if is_monster(picture_test, i, j):
                        check = True
                        for idx in idx_monster:
                            picture_test[i+idx[0]][j+idx[1]] = 'O'
                            
            if check:
                return picture_test

def count_sea_roughness(picture):
    return np.sum(picture == '#')

            
picture_tiles = place_tiles(dict_tiles)
picture = build_picture(picture_tiles)
solution = find_orientation(picture)
print(count_sea_roughness(solution))
print()
print_picture(solution)


2031

.......##..#...#.###........#.......#....#...###.....###..##..#........#...#.#...#.#..#.#.....#.
.#.....##.....##...#...##...#.#....#.##......#.#.#..#.#..#.......#.....O.#..........#..#.#...#..
.......#....#.#.##.##.###O#.....#.#....#..#..##.....#O....OO....OO..#.OOO...#....#...#...#......
#.##...O..#.OO##..OO#..#OOO......##...#......#........O##O..O..O#.O..O........#......###...#....
#.......O.#O..O#.O..O..O.#......##....###....##...#..#....#....#...#.........##.#..#...#........
........##..#.#.....#..##...##.#....#........#.#........#.......#...#...###.....##.....#..##....
##..#..........#...#........#..##.#.#.#.....#......##.......##O......#..##.#...#...#.....#O.#...
.###....................###.#.#.......#.....O.#..OO...#OO#.#.OOO.#.....#O...#OO.#.#OO...#OOO..##
.........#......#.#....#..#........#.#..##..#O.#O..O..O#.O..O#.##...#....O.#O.#O..O.#O..O..#....
#.#......#...#..........#...#...O...#....#...#.......#....#.#...........#.##..#.##.#...#..#.#...
....###...#...O.#.#OO.#.

.......#.#....#.....#...O....OO....OO#...OOO..#.O....OO...#OO....OOO.....#.....#.......#O.....#.
..##................#....O.#O.#O..O..O#.O.#......O.#O.#O..O..O#.O.....O....OO....OO....OOO.....#
##....#.........#..#..#.##.......#.##.#..#.#...#...#......##.#.#...#.##O..O..O#.O#.O..O#.#....#.
..#.##........#.#..O.......#....#....#.....##...#..........##...#.............#.#..##......#....
#O.#..OO....OO....OOO#.#...........#..##.###...#....##....#..##.#....#......#.#..#...##.#..#.#..
..O..O#.O..O..O..O...#.....#..#..#...#..#.#.......#.....##...#...#..#.#..#.#...#...........##..#
........#.#.##...................#......#.##...........#.......#.............##.........#.##....
..##..###......##...#...#....#....#.#..#..#....#.##............#...##....#.....#.#..#......O#...
.......#.#...#...#..........#.O..........#.....#....#.#.##.#......O.....#O#...OO.###OO....OOO#..
.....#...##.O#...OO..#.OO....OOO#.#.#.#.......#.O..#.OO..#.OO....OOO.....#O.#O#.O..O#.O#.O.....#
....#.#.#....O##O#.O..O##O#.O.